In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.impute import KNNImputer

from google.colab import files

In [ ]:
train = pd.read_csv('/content/hacktrain.csv')
test = pd.read_csv('/content/hacktest.csv')

print("Train shape:", train.shape)
print("Test shape:", test.shape)
train.head()

Train shape: (8000, 30)
Test shape: (2845, 29)


,Unnamed: 0,ID,class,20150720_N,20150602_N,20150517_N,20150501_N,20150415_N,20150330_N,20150314_N,...,20140610_N,20140525_N,20140509_N,20140423_N,20140407_N,20140322_N,20140218_N,20140202_N,20140117_N,20140101_N
0,0,1,water,637.5950,658.668,-1882.030,-1924.36,997.904,-1739.990,630.087,...,NaN,-1043.160,-1942.490,267.138,NaN,NaN,211.328,-2203.020,-1180.19,433.906
1,1,2,water,634.2400,593.705,-1625.790,-1672.32,914.198,-692.386,707.626,...,NaN,-933.934,-625.385,120.059,364.858,476.972,220.878,-2250.000,-1360.56,524.075
2,3,4,water,58.0174,-1599.160,NaN,-1052.63,NaN,-1564.630,NaN,...,-1025.88,368.622,NaN,-1227.800,304.621,NaN,369.214,-2202.120,NaN,-1343.550
3,4,5,water,72.5180,NaN,380.436,-1256.93,515.805,-1413.180,-802.942,...,-1813.95,155.624,NaN,-924.073,432.150,282.833,298.320,-2197.360,NaN,-826.727
4,7,8,water,1136.4400,NaN,NaN,1647.83,1935.800,NaN,2158.980,...,1535.00,1959.430,-279.317,-384.915,-113.406,1020.720,1660.650,-116.801,-568.05,-1357.140


In [ ]:
# Drop extra unnamed columns
train.drop(columns=[col for col in train.columns if "Unnamed" in col], inplace=True)
test.drop(columns=[col for col in test.columns if "Unnamed" in col], inplace=True)

# Fill missing NDVI values using KNN Imputer
ndvi_columns = [col for col in train.columns if '_N' in col]

imputer = KNNImputer(n_neighbors=5)
train[ndvi_columns] = imputer.fit_transform(train[ndvi_columns])
test[ndvi_columns] = imputer.transform(test[ndvi_columns])

In [ ]:
le = LabelEncoder()
train['class_encoded'] = le.fit_transform(train['class'])
print(dict(zip(le.classes_, le.transform(le.classes_))))  # See encoding

{'farm': np.int64(0), 'forest': np.int64(1), 'grass': np.int64(2), 'impervious': np.int64(3), 'orchard': np.int64(4), 'water': np.int64(5)}


In [ ]:
ndvi_columns = [col for col in train.columns if '_N' in col]

# Function to extract trend (slope) from time series
def calculate_trend(row):
    return np.polyfit(range(len(row)), row, 1)[0]

# Apply feature engineering
for df in [train, test]:
    df['ndvi_mean'] = df[ndvi_columns].mean(axis=1)
    df['ndvi_std'] = df[ndvi_columns].std(axis=1)
    df['ndvi_min'] = df[ndvi_columns].min(axis=1)
    df['ndvi_max'] = df[ndvi_columns].max(axis=1)
    df['ndvi_trend'] = df[ndvi_columns].apply(calculate_trend, axis=1)
    df['ndvi_skew'] = df[ndvi_columns].skew(axis=1)
    df['ndvi_kurtosis'] = df[ndvi_columns].kurtosis(axis=1)
    df['ndvi_median'] = df[ndvi_columns].median(axis=1)
    df['ndvi_range'] = df['ndvi_max'] - df['ndvi_min']
    df['ndvi_q25'] = df[ndvi_columns].quantile(0.25, axis=1)
    df['ndvi_q75'] = df[ndvi_columns].quantile(0.75, axis=1)

In [ ]:
features = ['ndvi_mean', 'ndvi_std', 'ndvi_min', 'ndvi_max', 'ndvi_trend', 'ndvi_skew',
            'ndvi_kurtosis','ndvi_range', 'ndvi_median', 'ndvi_q25', 'ndvi_q75']
X = train[features]
y = train['class_encoded']
X_test = test[features]

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

model = LogisticRegression(C=0.5, max_iter=1000, multi_class='multinomial', solver='lbfgs')
model.fit(X_scaled, y)

val_preds = model.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, val_preds))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Validation Accuracy: 0.879375


In [ ]:
test_preds = model.predict(X_test_scaled)
test_labels = le.inverse_transform(test_preds)

submission = pd.DataFrame({
    'ID': test['ID'],
    'class': test_labels
})

submission['ID'] = submission['ID'].astype(int)
submission['class'] = submission['class'].astype(str)
submission.to_csv('submission.csv', index=False, encoding='utf-8-sig',lineterminator='\n')
files.download('submission.csv')
print(submission.head())

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

   ID   class
0   1  forest
1   2  forest
2   3  forest
3   4  forest
4   5  forest
